In [1]:
import sqlite3
from faker import Faker
import random
import uuid

# Initialize Faker
faker = Faker()

# Connect to SQLite database (it will create a new database file)
conn = sqlite3.connect("students.db")
cursor = conn.cursor()

# Create Students Master Record Table
cursor.execute("""
CREATE TABLE IF NOT EXISTS StudentsMaster (
    StudentName TEXT,
    Gender TEXT,
    DOB TEXT,
    FatherName TEXT,
    MotherName TEXT,
    FathersOccupation TEXT,
    FathersIncome REAL,
    BloodGroup TEXT,
    Address TEXT,
    City TEXT,
    State TEXT,
    DateOfJoining TEXT,
    DateOfRecordCreation TEXT,
    UniqueStudentRegNo TEXT PRIMARY KEY
)
""")

# Create Students Marksheet Table
cursor.execute("""
CREATE TABLE IF NOT EXISTS StudentsMarksheet (
    UniqueStudentRegNo TEXT,
    Class TEXT,
    Section TEXT,
    TestTerm TEXT,
    Tamil INTEGER,
    English INTEGER,
    Maths INTEGER,
    Science INTEGER,
    SocialScience INTEGER,
    Total INTEGER,
    Average REAL,
    Grade TEXT,
    FOREIGN KEY (UniqueStudentRegNo) REFERENCES StudentsMaster (UniqueStudentRegNo)
)
""")

# Generate and Insert Fake Data
unique_reg_numbers = set()
students_master_data = []
students_marksheet_data = []

for _ in range(1000):
    unique_reg_no = str(uuid.uuid4())[:8]  # Generate unique reg number
    while unique_reg_no in unique_reg_numbers:
        unique_reg_no = str(uuid.uuid4())[:8]
    unique_reg_numbers.add(unique_reg_no)

    # Students Master Data
    student_data = (
        faker.name(),
        random.choice(["Male", "Female"]),
        str(faker.date_of_birth(minimum_age=6, maximum_age=18)),
        faker.name_male(),
        faker.name_female(),
        random.choice(["Engineer", "Teacher", "Doctor", "Farmer", "Businessman"]),
        round(random.uniform(50000, 500000), 2),
        random.choice(["A+", "A-", "B+", "B-", "O+", "O-", "AB+", "AB-"]),
        faker.address(),
        faker.city(),
        faker.state(),
        str(faker.date_between(start_date="-10y", end_date="today")),
        str(faker.date_this_year()),
        unique_reg_no
    )
    students_master_data.append(student_data)

    # Students Marksheet Data
    class_name = random.choice(["I", "II", "III", "IV", "V", "VI", "VII", "VIII", "IX", "X"])
    section = random.choice(["A", "B", "C", "D"])
    test_term = random.choice(["Term 1", "Term 2", "Term 3"])
    tamil = random.randint(35, 100)
    english = random.randint(35, 100)
    maths = random.randint(35, 100)
    science = random.randint(35, 100)
    social_science = random.randint(35, 100)
    total = tamil + english + maths + science + social_science
    average = total / 5
    grade = (
        "A" if average >= 80 else
        "B" if average >= 60 else
        "C" if average >= 50 else
        "D"
    )

    marksheet_data = (
        unique_reg_no,
        class_name,
        section,
        test_term,
        tamil,
        english,
        maths,
        science,
        social_science,
        total,
        round(average, 2),
        grade
    )
    students_marksheet_data.append(marksheet_data)

# Insert data into StudentsMaster and StudentsMarksheet tables
cursor.executemany("""
INSERT INTO StudentsMaster (
    StudentName, Gender, DOB, FatherName, MotherName, FathersOccupation,
    FathersIncome, BloodGroup, Address, City, State, DateOfJoining, DateOfRecordCreation, UniqueStudentRegNo
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""", students_master_data)

cursor.executemany("""
INSERT INTO StudentsMarksheet (
    UniqueStudentRegNo, Class, Section, TestTerm, Tamil, English, Maths,
    Science, SocialScience, Total, Average, Grade
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""", students_marksheet_data)

# Commit the changes
conn.commit()


import sqlite3
import pandas as pd

# Load data from both tables into pandas DataFrames
students_master_df = pd.read_sql_query("SELECT * FROM StudentsMaster", conn)
students_marksheet_df = pd.read_sql_query("SELECT * FROM StudentsMarksheet", conn)

# Perform the join operation using 'UniqueStudentRegNo' as the key
joined_df = pd.merge(students_master_df, students_marksheet_df, on='UniqueStudentRegNo', how='inner')

joined_df.columns = [s.lower() for s in joined_df.columns]

# Create a new table in SQLite from the joined DataFrame
joined_df.to_sql('JoinedStudents', conn, if_exists='replace', index=False)

df = pd.read_sql('select * from JoinedStudents', conn)

conn.close()

df

,studentname,gender,dob,fathername,mothername,fathersoccupation,fathersincome,bloodgroup,address,city,...,section,testterm,tamil,english,maths,science,socialscience,total,average,grade
0,Margaret Holt,Female,2016-12-23,Christopher Garrett,Tina Flynn,Teacher,74351.14,B-,"23643 Steve Burg\nPort Elizabethhaven, CA 32072",West Lisaberg,...,B,Term 2,93,50,94,52,67,356,71.2,B
1,Johnathan Lowery,Female,2011-02-24,Edward Green,Rachel Shepherd,Teacher,181432.97,A+,"536 Sean Springs Apt. 196\nGonzalezton, FL 18386",West Lisafort,...,A,Term 3,40,41,95,87,78,341,68.2,B
2,Sarah Crawford,Female,2009-06-29,Jesse Clark,Mary Watkins,Teacher,167658.16,B+,"94702 Lori Rue\nNicholsonborough, MP 47696",Whiteside,...,C,Term 3,84,98,81,64,38,365,73.0,B
3,Hailey Craig,Male,2018-09-09,Robert Mcfarland,Morgan Mahoney,Teacher,377735.20,AB+,"60394 Phillips Gateway Suite 251\nSouth Keith,...",Michaelchester,...,B,Term 1,52,63,92,46,54,307,61.4,B
4,Susan Watson,Male,2012-10-07,Bryan Williams,Mary Moore,Teacher,314049.99,AB+,"804 Jennifer Manor\nEast Jeanette, AZ 60693",Melissaberg,...,A,Term 1,76,49,76,91,84,376,75.2,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Maria Gonzalez,Female,2015-03-09,Scott Osborn,Leah Campos,Doctor,477326.22,A-,Unit 7867 Box 4738\nDPO AA 66879,Vickiberg,...,D,Term 3,85,92,56,48,97,378,75.6,B
1996,David Roberts,Female,2014-09-06,Stephen Christian,Gina Clark,Businessman,190656.06,B-,USS Joseph\nFPO AP 38107,Cooperside,...,A,Term 3,85,83,85,52,46,351,70.2,B
1997,Toni Hurst,Male,2015-07-09,Blake Pineda,Allison Castillo,Businessman,87147.04,AB+,"640 Davis Village Apt. 026\nWest David, OH 70817",Hudsonside,...,A,Term 1,71,37,93,36,40,277,55.4,C
1998,Leah Thompson,Female,2016-03-19,Bryce Wells,Sarah Larsen,Farmer,95278.61,B-,"735 Annette Throughway\nTeresashire, OR 22583",West Rhondamouth,...,B,Term 2,89,64,88,76,85,402,80.4,A
